In [3]:
import base64
import os

from dotenv import load_dotenv

from pydantic import BaseModel
from openai import OpenAI

load_dotenv()

openai_key = os.getenv('OPENAI_API_KEY')
openai_client = OpenAI()

class RequestBody(BaseModel):
    image: str

class MenuItemScored(BaseModel):
    name: str
    score: int
    red_flags: list[str]

class ResponseBody(BaseModel):
    item_list: list[MenuItemScored]

class MenuItem(BaseModel):
    name: str
    ingredients: list[str]

class Menu(BaseModel):
    choices: list[MenuItem]


In [4]:
def calculate_score(request: RequestBody):    
    completion = openai_client.chat.completions.create(
        model="gpt-4o",
        max_tokens=4096,
        temperature=0.0,
        messages=[
            {
                "role": "system",
                "content": f"""
                You are a culinary expert designed to extract food items and their ingredients from a menu.
                Please identify every menu item the image and provide a list of every ingredient listed on the menu for that item.
                If the ingredients are not listed, make an educated guess about what the ingredients are. Simplify the ingredients to raw materials.
                You must return a correctly formatted JSON object with the menu item name and the list of ingredients in the image and follows the schema below:

                JSON Schema:
                {Menu.model_json_schema()}""",
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": """Extract the products from the image,
                        and describe them in a query in JSON format""",
                    },
                    {
                        "type": "image_url", "image_url": {
                        "url": f"data:image/jpeg;base64,{request.image}"}},

                ],
            },
            {
                "role": "assistant",
                "content": "```json",
            },
        ],
    )
    print(completion)

    return completion


In [25]:
with open ("test_image_1.jpg", "rb") as image:
    image_data = base64.b64encode(image.read()).decode('utf-8')
    completion_message = calculate_score(RequestBody(image=image_data))

ChatCompletion(id='chatcmpl-A7fg3GbWr0KyhLlhhtGDTbVEjtANn', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "choices": [\n    {\n      "name": "Whipped Feta",\n      "ingredients": [\n        "Sweet and Hot Peppers",\n        "Seeded Crackers"\n      ]\n    },\n    {\n      "name": "Za’atar Bread",\n      "ingredients": [\n        "Pistachio Muhammara"\n      ]\n    },\n    {\n      "name": "Tahini Caesar",\n      "ingredients": [\n        "Grilled Corn",\n        "Cucumber",\n        "Crispy Basturma",\n        "Boquerones",\n        "Cornbread Croutons"\n      ]\n    },\n    {\n      "name": "Topik",\n      "ingredients": [\n        "Stuffed Potato Hummus",\n        "Pine Nuts",\n        "Currants",\n        "Tahini",\n        "Radish"\n      ]\n    },\n    {\n      "name": "Gigante Beans",\n      "ingredients": [\n        "Tomato",\n        "Cucumber",\n        "Turkish Green Olives",\n        "Orange-Coriander Vinaigrette"\n

In [26]:
completion_message

ChatCompletion(id='chatcmpl-A7fg3GbWr0KyhLlhhtGDTbVEjtANn', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "choices": [\n    {\n      "name": "Whipped Feta",\n      "ingredients": [\n        "Sweet and Hot Peppers",\n        "Seeded Crackers"\n      ]\n    },\n    {\n      "name": "Za’atar Bread",\n      "ingredients": [\n        "Pistachio Muhammara"\n      ]\n    },\n    {\n      "name": "Tahini Caesar",\n      "ingredients": [\n        "Grilled Corn",\n        "Cucumber",\n        "Crispy Basturma",\n        "Boquerones",\n        "Cornbread Croutons"\n      ]\n    },\n    {\n      "name": "Topik",\n      "ingredients": [\n        "Stuffed Potato Hummus",\n        "Pine Nuts",\n        "Currants",\n        "Tahini",\n        "Radish"\n      ]\n    },\n    {\n      "name": "Gigante Beans",\n      "ingredients": [\n        "Tomato",\n        "Cucumber",\n        "Turkish Green Olives",\n        "Orange-Coriander Vinaigrette"\n

In [27]:
import json

message_content = completion_message.choices[0].message.content
cleaned_content = message_content.replace('```json', '').replace('```', '').strip()
menu_dict = json.loads(cleaned_content)
menu = Menu(**menu_dict)
menu


Menu(choices=[MenuItem(name='Whipped Feta', ingredients=['Sweet and Hot Peppers', 'Seeded Crackers']), MenuItem(name='Za’atar Bread', ingredients=['Pistachio Muhammara']), MenuItem(name='Tahini Caesar', ingredients=['Grilled Corn', 'Cucumber', 'Crispy Basturma', 'Boquerones', 'Cornbread Croutons']), MenuItem(name='Topik', ingredients=['Stuffed Potato Hummus', 'Pine Nuts', 'Currants', 'Tahini', 'Radish']), MenuItem(name='Gigante Beans', ingredients=['Tomato', 'Cucumber', 'Turkish Green Olives', 'Orange-Coriander Vinaigrette']), MenuItem(name='Dark Leafy Greens', ingredients=['Tomato Sesame Jam']), MenuItem(name='Flattened Potatoes', ingredients=['Whipped Manouri Cheese', 'Dried Olives', 'Capers', 'Garlic Chips']), MenuItem(name='Red Pepper Dumplings', ingredients=['Tomato Brown Butter', 'Labne', 'Crispy Shallot', 'Nigella']), MenuItem(name='Fatteh', ingredients=['Caramelized Onion', 'Cauliflower', 'Crispy Mushrooms', 'Pine Nuts', 'Yogurt']), MenuItem(name='Feta Kataifi', ingredients=['S

In [28]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_openai import ChatOpenAI

class CarbonCost(BaseModel):
    carbon_cost: float
    explanation: str
    red_flags: list[str]
    score: int

open_langchain = ChatOpenAI(model="gpt-4o", temperature=0)

def calculate_carbon_cost(ingredient: str):
    system_prompt = f"""
    You are a environmental expert designed to calculate the carbon cost of food items and explain the carbon cost of the food item in pounds.
    Calculate the carbon cost of the food item in the message below and explain why the carbon cost is that value.
    Also provide a list of red flags that indicate that the production of the food item involves unsustainable practices such as human rights violations in third world countries, animal welfare concerns, etc.
    Finally, based on this information, provide a sustainability score for the food item on a scale of 1 to 10
    """

    chat_history = ChatMessageHistory()
    chat_history.add_message(SystemMessage(content=system_prompt))
    chat_history.add_message(HumanMessage(content=ingredient))

    prompt_template = ChatPromptTemplate.from_messages(
        [MessagesPlaceholder("messages"), SystemMessage(content=system_prompt)])
    runnable = prompt_template | open_langchain.with_structured_output(
        schema=CarbonCost, include_raw=True)
    
    carbon_cost = runnable.invoke({"messages": chat_history.messages})

    return carbon_cost


In [29]:
cost = calculate_carbon_cost(menu.choices[0].name)
cost['parsed']


CarbonCost(carbon_cost=3.5, explanation='Whipped feta is primarily made from feta cheese, which is a dairy product. Dairy production is known to have a significant carbon footprint due to the methane emissions from cows, the energy used in milking and processing, and the transportation of the final product. The carbon cost of dairy products like feta cheese is estimated to be around 3.5 pounds of CO2 per pound of cheese.', red_flags=['Animal welfare concerns related to dairy farming, including the treatment of cows.', 'High water usage in dairy farming, which can lead to water scarcity issues.', 'Potential use of antibiotics and hormones in dairy farming, which can have environmental and health impacts.', 'Human rights violations in some dairy-producing regions, including poor working conditions and low wages for farm workers.'], score=4)

In [21]:
cost = calculate_carbon_cost("steak")
cost['parsed']

CarbonCost(carbon_cost=39.2, explanation='The carbon cost of steak is approximately 39.2 pounds of CO2 equivalent per pound of beef. This high carbon cost is due to several factors: \n1. Methane Emissions: Cows produce methane, a potent greenhouse gas, during digestion. \n2. Land Use: Large areas of land are required for grazing and growing feed crops, leading to deforestation and loss of biodiversity. \n3. Water Use: Beef production is water-intensive, requiring thousands of gallons of water per pound of beef. \n4. Feed Production: Growing feed crops involves the use of fertilizers and pesticides, which contribute to greenhouse gas emissions.', red_flags=['Deforestation for grazing land', 'High water usage', 'Methane emissions from cattle', 'Use of antibiotics and hormones', 'Animal welfare concerns in feedlots', 'Human rights violations in feed production areas'], score=2)

In [24]:
cost = calculate_carbon_cost("milk")
cost['parsed']

CarbonCost(carbon_cost=4.8, explanation='The carbon cost of milk is approximately 4.8 pounds of CO2 equivalent per gallon. This value is derived from various factors including methane emissions from cows, energy used in milking and processing, and transportation. Methane is a potent greenhouse gas, and dairy cows produce a significant amount of it through enteric fermentation. Additionally, the feed for dairy cows often involves the use of fertilizers and pesticides, which also contribute to greenhouse gas emissions.', red_flags=['Animal welfare concerns: Dairy cows are often kept in confined spaces and may be subjected to practices like tail docking and dehorning without adequate pain relief.', 'Environmental degradation: Overgrazing and improper manure management can lead to soil degradation and water pollution.', 'Human rights violations: In some regions, dairy farm workers may face poor working conditions, low wages, and lack of labor rights.'], score=4)

In [23]:
cost = calculate_carbon_cost("potatoes")
cost['parsed']

CarbonCost(carbon_cost=0.31, explanation='Potatoes have a relatively low carbon footprint compared to many other food items. The carbon cost of potatoes is approximately 0.31 pounds of CO2 per pound of potatoes. This is because potatoes are a crop that requires relatively low amounts of water and fertilizer, and they have a high yield per acre. Additionally, they can be stored for long periods without significant energy inputs.', red_flags=['Monoculture farming practices can lead to soil degradation and loss of biodiversity.', 'Use of chemical fertilizers and pesticides can have negative environmental impacts.', 'Labor conditions in some potato farming regions may involve low wages and poor working conditions.'], score=8)